# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [1]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [3]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    pca = None
    scaler = None
    
    if hyperparams['use_pca']:
        # Standardizzazione da fare prima della PCA se richiesta
        if hyperparams['pca_standardize']:
            scaler = StandardScaler()  # creo lo scaler
            X_train = scaler.fit_transform(X_train)  # standardizzo i dati di training

        # creo l'oggetto PCA con un numero specificato di componenti, dopodichè applico la PCA
        pca = PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)

    # caso in cui non viene utilizzata la PCA e voglio solo fare la standardizzazione
    elif hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)

    # se utilizziamo la pca
    if pca is not None:
        # se abbiamo standardizzato prima della PCA
        if hyperparams['pca_standardize'] and scaler is not None:
            X_val = scaler.transform(X_val)
        X_val = pca.transform(X_val)
        
    # se abbiamo solo usato lo scaler (senza PCA)
    elif scaler is not None:
        X_val = scaler.transform(X_val)
        
    # Aggiunge il termine costante ai dati
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        lambda_reg = hyperparams['reg_lambda']
        I = np.eye(X_train.shape[1])
        # escludo l'intercetta nella regolarizzazione
        I[-1, -1] = 0
        params = np.linalg.inv(X_train.T @ X_train + lambda_reg * I) @ X_train.T @ y_train
    else:
        params = np.linalg.pinv(X_train) @ y_train  # Soluzione ai minimi quadrati

    # Calcolo predizioni
    y_train_pred = X_train @ params
    y_val_pred = X_val @ params

    # Calcola il MAE
    mae_train = np.mean(np.abs(y_train - y_train_pred))  # MAE per il training set
    mae_val = np.mean(np.abs(y_val - y_val_pred))  # MAE per il validation set

    return params, mae_train, mae_val

def test_pipeline(X, y, hyperparams):
    # Dividi il dataset in training e test set
    train_fraction = 0.8  # 80% training+validation, 20% test
    num_train_samples = int(train_fraction * X.shape[0])  # Calcolo numero campioni training

    # Divisione dataset
    X_main = X[:num_train_samples]  # Dati principali (train+val)
    y_main = y[:num_train_samples]
    X_final_test = X[num_train_samples:]  # Dati di test finale
    y_final_test = y[num_train_samples:]

    print(f"Dataset split: {X_main.shape[0]} train/val | {X_final_test.shape[0]} test")

    # Dividi il training set in training set effettivo e validation set
    train_samples = int(0.8 * X_main.shape[0])

    X_model_training = X_main[:train_samples]
    y_model_training = y_main[:train_samples]
    X_validation = X_main[train_samples:]
    y_validation = y_main[train_samples:]

    print("[DIMENSIONI DATASET]")
    print(f"Training: {X_model_training.shape} ({(X_model_training.shape[0]/X.shape[0])*100:.1f}% del totale)")
    print(f"Validation: {X_validation.shape} ({(X_validation.shape[0]/X.shape[0])*100:.1f}%)") 
    print(f"Test: {X_final_test.shape} ({(X_final_test.shape[0]/X.shape[0])*100:.1f}%)")
    print(f"Totale campioni verificato: {X_final_test.shape[0] + X_model_training.shape[0] + X_validation.shape[0]}")

    # Trova la configurazione di iperparametri migliore
    best_mae_val = float('inf')
    best_config = None
    best_mae_train = None

    for config in hyperparams:
        X_temp = X_model_training.copy()
        Xv_temp = X_validation.copy()

        _, mae_train, mae_val = pipeline(X_temp, y_model_training, 
                                      Xv_temp, y_validation, config)

        if mae_val < best_mae_val:
            best_mae_val = mae_val
            best_mae_train = mae_train
            best_config = config

    print("\nMiglior risultato trovato:")
    print("Configurazione:", best_config)
    print("MAE Training:", round(best_mae_train, 4))
    print("MAE Validation:", round(best_mae_val, 4))

    return best_config, best_mae_train, best_mae_val

def pipeline_best_mae(X, y, hyperparams):
    # Divisione del dataset in training (80%) e test (20%)
    train_fraction = 0.8
    num_train_samples = int(train_fraction * X.shape[0])

    X_train_full = X[:num_train_samples]
    y_train_full = y[:num_train_samples]
    X_test_final = X[num_train_samples:]
    y_test_final = y[num_train_samples:]

    # Riallena il modello sul training set completo
    best_config, best_mae_train, best_mae_val = test_pipeline(X, y, hyperparams)

    # Calcola il MAE sul test set
    _, mae_train, mae_test = pipeline(X_train_full, y_train_full, X_test_final, y_test_final, best_config)

    return best_config, best_mae_train, best_mae_val, mae_test

# Stampa risultati
config, mae_train, mae_val, mae_test = pipeline_best_mae(X, y, configs)

print("\nRisultati:")
print("Configurazione:", config)
print("MAE Train:", mae_train)
print("MAE Val:", mae_val)
print("MAE Test:", mae_test)

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Dataset split: 3918 train/val | 980 test
[DIMENSIONI DATASET]
Training: (3134, 11) (64.0% del totale)
Validation: (784, 11) (16.0%)
Test: (980, 11) (20.0%)
Totale campioni verificato: 4898

Miglior risultato trovato:
Configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': False, 'use_regularization': False, 'reg_lambda': None}
MAE Training: 0.606
MAE Validation: 0.5737

Risultati:
Configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': False, 'use_regularization': False, 'reg_lambda': None}
MAE Train: 0.6060289033176667
MAE Val: 0.573663326947442
MAE Test: 0.5483541653889216
